In [1]:
# importing the requests library
import requests
import geocoder
import mysql.connector

In [102]:
dbip="localhost"
#database creation
#incdb = mysql.connector.connect(
#  host=dbip,
#  user="user",
#  password="1234"
#)
#inccursor = incdb.cursor()
#inccursor.execute("CREATE DATABASE incdatabase")

#check database
#inccursor.execute("SHOW DATABASES")
#for x in inccursor:
#  print(x)

#create table
#incdb = mysql.connector.connect(
#  host=dbip,
#  user="user",
#  password="1234",
#  database="incdatabase"
#)

inccursor = incdb.cursor()


#inccursor.execute("CREATE TABLE incident (id CHAR(255), severity CHAR(255), type CHAR(255), shortDesc CHAR(255), lat CHAR(255), lng CHAR(255))")


incdb = mysql.connector.connect(
  host=dbip,
  user="user",
  password="1234",
  database="incdatabase"
)

inccursor = incdb.cursor()
#ahol nincs id
sql = "INSERT INTO incident (id, severity, type, shortDesc, lat, lng ) VALUES (%s, %s, %s, %s, %s, %s)"
val = (410226587897614656, 1, 1, "Road construction",47.54751996137202,19.262960040941834)
inccursor.execute(sql, val)

incdb.commit()


In [74]:
def storeIncidents(a, b, c, d):

    incdb = mysql.connector.connect(
    host=dbip,
    user="user",
    password="1234",
    database="incdatabase"
    )
    inccursor = incdb.cursor()
    
    inccursor.execute("SELECT id FROM incident")

    storedincidents = inccursor.fetchall()
    
    sql = "INSERT INTO incident (id, severity, type, shortDesc, lat, lng ) VALUES (%s, %s, %s, %s, %s, %s)"
    
    URL = "http://www.mapquestapi.com/traffic/v2/incidents"
    mapquestKey = "BGFBW3Gk3NZNBbDdk6MUM3zEoadG1lDC"
    #budapest
    Box = "{},{},{},{}".format(a,b,c,d)
    Filter = "construction,incidents"

    # defining a params dict for the parameters to be sent to the API
    PARAMS = {'key':mapquestKey,'boundingBox':Box,'filters':Filter}

    # sending get request and saving the response as response object
    r = requests.get(url = URL, params = PARAMS)

    # extracting data in json format
    data = r.json()
    for inc in data['incidents']:
        found = 0
        for stored in storedincidents:
            if stored[0].find(inc['id']) != -1:

                found = 1
        if found == 0:
            val = (inc['id'], inc['severity'], inc['type'], inc['shortDesc'],inc['lat'],inc['lng'])
            inccursor.execute(sql, val)
    incdb.commit()
    return data

incidents = storeIncidents(47.35, 18.8, 47.60, 19.4)

#for inc in incidents['incidents']:
#    print(inc['severity'])


In [106]:
def samepoint(x1,x2,y1,y2):
    if abs(float(x1)-float(x2)) < 0.001 and abs(float(y1)-float(y2)) < 0.001:
        return True
    else:
        return False

def StoredIncidents():
    incdb = mysql.connector.connect(
    host=dbip,
    user="user",
    password="1234",
    database="incdatabase"
    )
    inccursor = incdb.cursor()

    inccursor.execute("SELECT lat, lng, severity FROM incident ORDER BY lat")
    
    storedincidents = inccursor.fetchall()
    x = 0 
    list = []
    while x < len(storedincidents):
        sev = int(storedincidents[x][2])
        i = 1
        samepotential = True
        while x+i < len(storedincidents) and samepotential:
            if samepoint(storedincidents[x][0],storedincidents[x+i][0],storedincidents[x][1],storedincidents[x+i][1]):
                sev += int(storedincidents[x+i][2])
                i+=1
            else:
                samepotential = False
        list.append(tuple((storedincidents[x][0], storedincidents[x][1],sev)))
        x+=i
    return list

asd = StoredIncidents()

[('47.31809997931123', '19.171799970790744', 1),
 ('47.35108000226319', '19.197569964453578', 1),
 ('47.373200012370944', '19.025139966979623', 6),
 ('47.37554996274412', '19.022369999438524', 3),
 ('47.384269991889596', '19.112070025876164', 3),
 ('47.385749984532595', '19.11027998663485', 3),
 ('47.399950018152595', '18.972700014710426', 1),
 ('47.41348000243306', '19.315880024805665', 1),
 ('47.41890996694565', '18.892600033432245', 1),
 ('47.42126997560263', '18.894250011071563', 1),
 ('47.423220025375485', '19.105329969897866', 3),
 ('47.43120001628995', '18.898450015112758', 1),
 ('47.43455001153052', '18.902220027521253', 1),
 ('47.43514001369476', '19.072970040142536', 3),
 ('47.43554997257888', '18.904629992321134', 1),
 ('47.43688001297414', '18.902039984241128', 3),
 ('47.44194000028074', '19.080160036683083', 3),
 ('47.44255003519356', '19.07213998027146', 1),
 ('47.45687998831272', '19.0505900233984', 1),
 ('47.45710998773575', '19.04836001805961', 1),
 ('47.45789998210966